In [126]:
import pandas as pd
%matplotlib inline
import pylab as plt
import numpy as np
import scipy as sc
import scipy.stats

# High school test scores data

Since our correlation metrics -- as predicted -- did not show high enough significance between the GPA and the yield, we decided to add new features to our datasets.

Fortunately, we have found a dataset with California SAT, ACT and AP test scores by year and high school: https://www.cde.ca.gov/ds/sp/ai/. The dataset consists of multiple `.xls` files grouped by:
 - year
 - test
 
Moreover, the `.xls` files have different formats. Examples of differences include:
 - extra/missing columns
 - number of rows before the header
 - different column names
 
We prepared the following functions to resolve these differences:

In [127]:
def read_multi_xls(prefix, start_year, end_year, skip_row_count=0):
    data_frames = []
    for year in range(start_year, end_year+1):
        year_2dig = year % 100
        df = pd.read_excel(prefix + str(year_2dig).zfill(2) + '.xls',
                skiprows=list(range(skip_row_count)))
        df['year'] = year
        data_frames.append(df)
    return data_frames

def import_multi_xls(prefix, start_year, end_year, skip_row_count, columns_to_remove, must_have_columns=[]):
    data_frames = []
    column_names = None
    for year in range(start_year, end_year+1):
        year_2dig = year % 100
        df = pd.read_excel(prefix + str(year_2dig).zfill(2) + '.xls',
                skiprows=list(range(skip_row_count[year])))
        for c in columns_to_remove[year]:
            del df[c]
        
        # There are differences between column names, so we use the names from the first
        # dataframe
        if column_names is None:
            column_names = list(df)
        
        for col in must_have_columns:
            if col not in list(df):
                df.insert(column_names.index(col), col, np.nan)
                
        new_column_names = list(df)
        
        renamer = {}
        for i in range(len(column_names)):
            renamer[new_column_names[i]] = column_names[i]
        df.rename(columns=renamer, inplace=True)
        
        df['year'] = year
        
        data_frames.append(df)
    return pd.concat(data_frames)

## AP Scores

The first set of scores we will import is the AP scores. It turns out that the data between 1999 and 2013 has a similar format, and we should be able to perform a single `import_multi_xls()` call for all of it.

In [128]:
ap_99_13 = import_multi_xls('data/test_scores/ap/ap', 1999, 2013, {
        1999: 2, 2000: 2, 2001: 2, 2002: 2, 2003: 2, 2004: 2, 2005: 2, 2006: 2, 2007: 2, 2008: 2,
        2009: 3, 2010: 3, 2011: 3, 2012: 3, 2013: 3,
    }, {
        1999: [], 2000: [], 2001: [], 2002: [], 2003: [],
        2004: ['Rate of Exams\nWith a Score of\n3 or Greater\nFor 12th Grade\nStudents', 'Rate of Exams\nWith a Score of\n3 or Greater\nFor 11th & 12th\nGrade Students'],
        2005: [], 2006: [], 2007: [], 2008: [], 2009: [], 2010: ['Year'], 2011: [], 2012: [], 2013: [],
    },
    ['Total\nNumber of\nExams Taken', 'Number\nof Exams\nWith a Score of\n3 or Greater']
)
ap_99_13.head()

,County Number,District Number,School Number,County Name,District Name,School Name,Grade 11 Enrollment (October 1998 CBEDS),Grade 12 Enrollment (October 1998 CBEDS),Grade 11+12 Enrollment (October 1998 CBEDS),Number of AP Exam Takers,Total Number of Exams Taken,Number of Exams With a Score of 3 or Greater,Number of Exams With a Score of 1,Number of Exams With a Score of 2,Number of Exams With a Score of 3,Number of Exams With a Score of 4,Number of Exams With a Score of 5,year
0,1,10017,130401,Alameda,Alameda Co. Office Of Educatio,Juvenile Hall/Court,157.000000,224.000000,381.000000,0.000000,0,0,0,0,0,0,0,1999
1,1,10017,130419,Alameda,Alameda Co. Office Of Educatio,County Community,3.000000,8.000000,11.000000,0.000000,0,0,0,0,0,0,0,1999
2,1,10017,130427,Alameda,Alameda Co. Office Of Educatio,Alternative/Opportunity,35.000000,67.000000,102.000000,0.000000,0,0,0,0,0,0,0,1999
3,1,61119,130229,Alameda,Alameda City Unified,Alameda High,413.000000,372.000000,785.000000,165.000000,272,183,35,54,95,54,34,1999
4,1,61119,132878,Alameda,Alameda City Unified,Encinal High,350.000000,240.000000,590.000000,138.000000,267,77,86,104,53,22,2,1999


The data from 2014 to 2016 is formatted differently, and we will need to import it separately:

In [129]:
ap_14_16_dfs = read_multi_xls('data/test_scores/ap/ap', 2014, 2016)
for df in ap_14_16_dfs:
    # There are some lower/upper case differences in column names which causes problems
    # while merging.
    df.columns = map(str.lower, df.columns)
ap_14_16 = pd.concat(ap_14_16_dfs)
ap_14_16.head()

,cds,cname,dname,enroll1012,enroll12,numscr1,numscr2,numscr3,numscr4,numscr5,numtsttakr,rtype,sname,year
0,1612000133397,Alameda,Livermore Valley Joint Unified ...,1476,467,38,100,220,197,115,394,S,Granada High ...,2014
1,1612000134536,Alameda,Livermore Valley Joint Unified ...,1294,438,37,87,156,98,62,255,S,Livermore High ...,2014
2,1612340000000,Alameda,Newark Unified ...,1539,491,117,181,172,103,63,324,D,...,2014
3,1612340130054,Alameda,Newark Unified ...,1414,418,117,181,172,103,63,324,S,Newark Memorial High ...,2014
4,1612340130484,Alameda,Newark Unified ...,33,14,NaN,NaN,NaN,NaN,NaN,0,S,Crossroads High (Alternative) ...,2014


We will also extract the school number from the CDS number:

In [130]:
ap_14_16['school_num'] = pd.to_numeric(ap_14_16['cds'].astype(str).str[-7:])
ap_14_16.head()

,cds,cname,dname,enroll1012,enroll12,numscr1,numscr2,numscr3,numscr4,numscr5,numtsttakr,rtype,sname,year,school_num
0,1612000133397,Alameda,Livermore Valley Joint Unified ...,1476,467,38,100,220,197,115,394,S,Granada High ...,2014,133397
1,1612000134536,Alameda,Livermore Valley Joint Unified ...,1294,438,37,87,156,98,62,255,S,Livermore High ...,2014,134536
2,1612340000000,Alameda,Newark Unified ...,1539,491,117,181,172,103,63,324,D,...,2014,0
3,1612340130054,Alameda,Newark Unified ...,1414,418,117,181,172,103,63,324,S,Newark Memorial High ...,2014,130054
4,1612340130484,Alameda,Newark Unified ...,33,14,NaN,NaN,NaN,NaN,NaN,0,S,Crossroads High (Alternative) ...,2014,130484


Also, we want to drop the cumulative data:

In [131]:
ap_14_16 = ap_14_16[ap_14_16['rtype'] == 'S']
ap_14_16.head()

,cds,cname,dname,enroll1012,enroll12,numscr1,numscr2,numscr3,numscr4,numscr5,numtsttakr,rtype,sname,year,school_num
0,1612000133397,Alameda,Livermore Valley Joint Unified ...,1476,467,38,100,220,197,115,394,S,Granada High ...,2014,133397
1,1612000134536,Alameda,Livermore Valley Joint Unified ...,1294,438,37,87,156,98,62,255,S,Livermore High ...,2014,134536
3,1612340130054,Alameda,Newark Unified ...,1414,418,117,181,172,103,63,324,S,Newark Memorial High ...,2014,130054
4,1612340130484,Alameda,Newark Unified ...,33,14,NaN,NaN,NaN,NaN,NaN,0,S,Crossroads High (Alternative) ...,2014,130484
6,1612420126763,Alameda,New Haven Unified ...,125,73,NaN,NaN,NaN,NaN,NaN,0,S,Decoto School for Independent Study ...,2014,126763


Next, we will rename and drop some of the columns in our dataframes.

In [132]:
ap_14_16.drop(columns=[
    'cds', 'cname', 'dname', 'rtype', 'enroll1012'
], inplace=True)
ap_99_13.drop(columns=[
    'County\nNumber', 'District\nNumber', 'County Name', 'District Name',
    'Total\nNumber of\nExams Taken', 'Number\nof Exams\nWith a Score of\n3 or Greater',
    'Grade 11\nEnrollment\n(October 1998\nCBEDS)',
    'Grade 11+12\nEnrollment\n(October 1998\nCBEDS)',
], inplace=True)

ap_99_13_renamer = {
    'School\nNumber': 'school_num',
    'School Name': 'school_name',
    'Number of\nAP Exam\nTakers': 'ap_num_test_takers',
    'Number\nof Exams\nWith a Score of\n1': 'ap_num_scr1',
    'Number\nof Exams\nWith a Score of\n2': 'ap_num_scr2',
    'Number\nof Exams\nWith a Score of\n3': 'ap_num_scr3',
    'Number\nof Exams\nWith a Score of\n4': 'ap_num_scr4',
    'Number\nof Exams\nWith a Score of\n5': 'ap_num_scr5',
    'Grade 12\nEnrollment\n(October 1998\nCBEDS)': 'enroll12',
    'year': 'year',
}
ap_99_13.rename(columns=ap_99_13_renamer, inplace=True)

ap_14_16_renamer = {
    'school_num': 'school_num',
    'sname': 'school_name',
    'numtsttakr': 'ap_num_test_takers',
    'numscr1': 'ap_num_scr1',
    'numscr2': 'ap_num_scr2',
    'numscr3': 'ap_num_scr3',
    'numscr4': 'ap_num_scr4',
    'numscr5': 'ap_num_scr5',
    'enroll12': 'enroll12',
    'year': 'year',
}
ap_14_16.rename(columns=ap_14_16_renamer, inplace=True)

Finally, we merge the datasets

In [133]:
ap_scores = pd.concat([ap_99_13, ap_14_16])

All of our columns should have numeric values at this point (apart from the school_name). We will make pandas convert all values to numeric (and all non-number values to NaN)

In [134]:
school_names = np.copy(ap_scores['school_name'])
ap_scores = ap_scores.apply(pd.to_numeric, errors='coerce', axis=1)
ap_scores['school_name'] = school_names

The last step will be to modify the year column. Our main dataset uses the starting year of the academic year, whereas our `ap_scores` dataframe uses the ending year of the academic year.

In [135]:
ap_scores['year'] = ap_scores['year'] - 1
ap_scores

,ap_num_scr1,ap_num_scr2,ap_num_scr3,ap_num_scr4,ap_num_scr5,ap_num_test_takers,enroll12,school_name,school_num,year
0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,224.000000,Juvenile Hall/Court,130401.000000,1998.000000
1,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,8.000000,County Community,130419.000000,1998.000000
2,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,67.000000,Alternative/Opportunity,130427.000000,1998.000000
3,35.000000,54.000000,95.000000,54.000000,34.000000,165.000000,372.000000,Alameda High,130229.000000,1998.000000
4,86.000000,104.000000,53.000000,22.000000,2.000000,138.000000,240.000000,Encinal High,132878.000000,1998.000000
5,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,119.000000,Island High (Cont.),134304.000000,1998.000000
6,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,23.000000,Macgregor High (Cont.),130294.000000,1998.000000
7,12.000000,22.000000,39.000000,25.000000,30.000000,89.000000,184.000000,Albany High,130450.000000,1998.000000
8,25.000000,46.000000,90.000000,130.000000,223.000000,325.000000,687.000000,Berkeley High,131177.000000,1998.000000
9,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,44.000000,"East Campus, Berkeley Hig",134924.000000,1998.000000


We are saving the processed data to a CSV file. This way we will not have to redo all of our computations.

In [136]:
ap_scores.to_csv('data/test_scores/ap/processed.csv', sep=',', index=False)

## SAT scores

Similarly to the AP scores, SAT scores are also split by year, and the .xls files have different formats. We will need to perform merging similar to the one in the *AP scores* section. 

In [137]:
def import_sat_xls(prefix, start_year, end_year, columns, skip_row_count):
    data_frames = []
    for year in range(start_year, end_year+1):
        year_2dig = year % 100
        df = pd.read_excel(prefix + str(year_2dig).zfill(2) + '.xls',
                skiprows=list(range(skip_row_count[year])))
        df = df[columns]
        df['year'] = year
        data_frames.append(df)
    return pd.concat(data_frames)

In 2006, SAT scores changed from having two categories (verbal, math) to three categories (reading, writing, math). This will make our format differ slightly.

In [138]:
sat_99_05 = import_sat_xls(
    'data/test_scores/sat/sat', 1999, 2005,
    ['School\nNumber', 'Number\nof\nTakers', 'Average\nVerbal\nScore',
    'Average\nMath\nScore', 'Average\nTotal\nScore'],
    {
        1999: 2, 2000: 2, 2001: 2, 2002: 2, 2003: 2,
        2004: 2, 2005: 2,
    }
)

In [139]:
sat_06_07 = import_sat_xls(
    'data/test_scores/sat/sat', 2006, 2007,
    ['School\nNumber', 'Number\nof\nTakers', 'Average\nVerbal\nScore',
    'Average\nMath\nScore', 'Average\nWriting\nScore', 'Average\nTotal\nScore'],
    {
        2006: 2, 2007: 2,
    }
)

In [140]:
sat_08_10 = import_sat_xls(
    'data/test_scores/sat/sat', 2008, 2010,
    ['School\nNumber', 'Number\nTested', '\nCritical Reading\nAverage',
    '\nMath\nAverage', '\nWriting\nAverage', '\nTotal\nAverage'],
    {
        2008: 2, 2009: 3, 2010: 4
    }
)

sat_11_13 = import_sat_xls(
    'data/test_scores/sat/sat', 2011, 2013,
    ['School\nNumber', 'Number\nTested', 'V_Mean',
    'M_Mean', 'W_Mean', 'Tot_Mean'],
    {
        2011: 3, 2012: 3, 2013: 3
    }
)
sat_14_16 = pd.concat(read_multi_xls('data/test_scores/sat/sat', 2014, 2016))

We need to know the school number in the `sat_14_16` dataframe. We will derive it from the `cds` columns:

In [141]:
sat_14_16['school_num'] = pd.to_numeric(sat_14_16['cds'].astype('int64').astype(str).str[-7:])

In order to do some processing on the scores, we will convert the dataframes to a numeric format.

In [142]:
sat_14_16 = sat_14_16.apply(pd.to_numeric, errors='coerce', axis=1)
sat_11_13 = sat_11_13.apply(pd.to_numeric, errors='coerce', axis=1)
sat_08_10 = sat_08_10.apply(pd.to_numeric, errors='coerce', axis=1)
sat_99_05 = sat_99_05.apply(pd.to_numeric, errors='coerce', axis=1)
sat_06_07 = sat_06_07.apply(pd.to_numeric, errors='coerce', axis=1)

Now, we will merge the dataframes into a single dataframe. To keep the format constant between the years we will combine the *reading* and *writing* scores into a single *verbal* score. We will also modify the *total score* accordingly.

In [143]:
sat_06_07['sat_verbal_avg'] = (sat_06_07['Average\nWriting\nScore'] + sat_06_07['Average\nVerbal\nScore'])/2
sat_06_07.drop(columns=['Average\nWriting\nScore', 'Average\nVerbal\nScore'], inplace=True)

sat_08_10['sat_verbal_avg'] = (sat_08_10['\nWriting\nAverage'] + sat_08_10['\nCritical Reading\nAverage'])/2
sat_08_10.drop(columns=['\nWriting\nAverage', '\nCritical Reading\nAverage'], inplace=True)

sat_11_13['sat_verbal_avg'] = (sat_11_13['W_Mean'] + sat_11_13['V_Mean'])/2
sat_11_13.drop(columns=['W_Mean', 'V_Mean'], inplace=True)

sat_14_16['sat_verbal_avg'] = (sat_14_16['AvgScrRead'] + sat_14_16['AvgScrWrite'])/2
sat_14_16['sat_total_avg'] = (
    sat_14_16['AvgScrRead'] +
    sat_14_16['AvgScrWrite'] +
    sat_14_16['AvgScrMath'])*(2/3)
sat_14_16 = sat_14_16[['school_num', 'NumTstTakr', 'sat_verbal_avg', 'AvgScrMath',
                      'sat_total_avg', 'year']]

sat_08_10['\nTotal\nAverage'] = sat_08_10['\nTotal\nAverage'] * (2/3)
sat_06_07['Average\nTotal\nScore'] = sat_06_07['Average\nTotal\nScore'] * (2/3)
sat_11_13['Tot_Mean'] = sat_11_13['Tot_Mean'] * (2/3)

sat_99_07_renamer = {
    'School\nNumber': 'school_num',
    'Number\nof\nTakers': 'sat_num_test_takers',
    'Average\nVerbal\nScore': 'sat_verbal_avg',
    'Average\nMath\nScore': 'sat_math_avg',
    'Average\nTotal\nScore': 'sat_total_avg',
    'year': 'year'
}
sat_08_10_renamer = {
    'School\nNumber': 'school_num',
    'Number\nTested': 'sat_num_test_takers',
    '\nMath\nAverage': 'sat_math_avg',
    '\nTotal\nAverage': 'sat_total_avg',
    'sat_verbal_avg': 'sat_verbal_avg',
    'year': 'year',
}
sat_11_13_renamer = {
    'School\nNumber': 'school_num',
    'Number\nTested': 'sat_num_test_takers',
    'M_Mean': 'sat_math_avg',
    'Tot_Mean': 'sat_total_avg',
    'sat_verbal_avg': 'sat_verbal_avg',
    'year': 'year',
}
sat_14_16_renamer = {
    'NumTstTakr': 'sat_num_test_takers',
    'AvgScrMath': 'sat_math_avg',
}

In [144]:
sat_99_05.rename(columns=sat_99_07_renamer, inplace=True)
sat_06_07.rename(columns=sat_99_07_renamer, inplace=True)
sat_08_10.rename(columns=sat_08_10_renamer, inplace=True)
sat_11_13.rename(columns=sat_11_13_renamer, inplace=True)
sat_14_16.rename(columns=sat_14_16_renamer, inplace=True)

In [145]:
sat_scores = pd.concat([sat_99_05, sat_06_07, sat_08_10, sat_11_13, sat_14_16])

Next, we need to drop the culative rows (where `school_num` is 0), and update the `year` field to reflect the format of our main dataset.

We will also mark 0 scores as NaN.

In [146]:
sat_scores = sat_scores[sat_scores['school_num'] != 0]
sat_scores['year'] = sat_scores['year'] - 1
sat_scores = sat_scores.replace(0, np.nan)

Lastly, we will save the data into a CSV file so that we don't have to rerun our preprocessing.

In [147]:
sat_scores.to_csv('data/test_scores/sat/processed.csv', sep=',', index=False)